In [1]:
#merge the two data frames according to item number 
import pandas as pd
unclean_df = pd.read_csv('/Users/chenyihe/Desktop/eBay Data/EbayPcLaptopDataUnclean.csv')
price_df = pd.read_csv('/Users/chenyihe/Desktop/eBay Data/EbayPcLaptopPriceData.csv')
merged_df = pd.merge(unclean_df, price_df, on='Item Number', how='inner')


In [2]:
# Calculate the percentage of missing values for each row
missing_percentage_per_row = merged_df.isnull().mean(axis=1) * 100

#drop the row with more than 80% missing value
merged_df = merged_df[merged_df.isnull().mean(axis=1) < 0.8]

missing_percentage_per_row

0       16.666667
1       25.000000
2       12.500000
3       25.000000
4       20.833333
          ...    
6843    87.500000
6844    79.166667
6845    87.500000
6846    79.166667
6847    87.500000
Length: 6848, dtype: float64

In [3]:
#calculate missing percentage of each column 
missing_value_counts = merged_df.isna().sum()
missing_value_percentages = missing_value_counts/len(merged_df) * 100
missing_value_percentages


Item Number                       0.000000
Brand                            11.221681
Rating                           93.626932
Ratings Count                    93.626932
Processor                        13.762439
Screen Size                      12.597925
Manufacturer Color               95.765403
Color                            52.085539
Ram Size                         51.408003
SSD Capacity                     36.862164
GPU                              38.810078
Processor Speed                  39.635825
Type                             26.720305
Release Year                     81.621851
Maximum Resolution               44.145670
Model                            28.879949
OS                               31.420707
Hard Drive Capacity              47.745077
Country Region Of Manufacture    95.574846
Storage Type                     45.775990
Features                         43.446962
Condition                         9.781918
Seller Note                      69.447385
Price      

In [4]:
#fill missing value in columns with under 10% missing value, and drop the column with more than 60% missing value
#since the Ratings, Rating Count, Manufacturer Color and Country Region of Manufacture columns has over 70% of missing value, we can drop these columns 
columns_to_drop = ['Rating', 'Ratings Count', 'Manufacturer Color', 'Country Region Of Manufacture', 'Release Year', 'Seller Note']

#record the cleaned data frame to a new data frame
cleaned_df = merged_df.drop(columns=columns_to_drop)

# Clean the Condition column

# Since the Conditoin column only has 9% of missing value, we can fill those missing condition with "Unknown"
#condition values vary a lot and might be useful for customer therefore we can just leave them there
cleaned_df['Condition'] = cleaned_df['Condition'].fillna('Unknown')

missing_value_counts = cleaned_df.isna().sum()
missing_value_percentages = missing_value_counts/len(cleaned_df) * 100
missing_value_percentages

Item Number             0.000000
Brand                  11.221681
Processor              13.762439
Screen Size            12.597925
Color                  52.085539
Ram Size               51.408003
SSD Capacity           36.862164
GPU                    38.810078
Processor Speed        39.635825
Type                   26.720305
Maximum Resolution     44.145670
Model                  28.879949
OS                     31.420707
Hard Drive Capacity    47.745077
Storage Type           45.775990
Features               43.446962
Condition               0.000000
Price                   0.000000
dtype: float64

In [5]:
# Clean the Brand column

# Browse all the values of brand
brand_values = cleaned_df['Brand'].unique()

# Standardize the 'Brand' column by removing leading/trailing spaces and converting to proper case
cleaned_df['Brand'] = cleaned_df['Brand'].str.strip().str.title()

# Replace any missing or NaN values in the 'Brand' column with "Unknown"
cleaned_df['Brand'] = cleaned_df['Brand'].fillna('Unknown')

#Replace "Shed Number 2, ?, Does not apply, Not applicable, Unbranded, Www.Tag1000Diver.Com, Top Performance, See Title/Description, Unbranded/Generic, Major Brand" values in the 'Brand' column with "Unknown"
values_to_replace = [
    'Shed Number 2', '?', 'Not Applicable', 'Does Not Apply', 'Unbranded',
    'Www.Tag1000Diver.Com', 'Top Performance', 'See Title/Description', 
    'Unbranded/Generic', 'Major Brand'
]

cleaned_df['Brand'] = cleaned_df['Brand'].replace(values_to_replace, 'Unknown')

# Correct the missspelling name
brand_replacements = {
    'Chungwa': 'Chunghwa',
    'Chingwa': 'Chunghwa',
    'Lenova': 'Lenovo'
}

cleaned_df['Brand'] = cleaned_df['Brand'].replace(brand_replacements)

# Correct the version types to brand names 
brand_replacements = {
    'Claa156Wa13A': 'CPT',
    'Claa156Wa11A': 'CPT',
    'Claa116Wa03A': 'CPT',
    'Hw13Wx001': 'BoeHydis',
    'C600': 'Dell'
}

cleaned_df['Brand'] = cleaned_df['Brand'].replace(brand_replacements)

# If the brand is not sure and have several choice such as "Delta / Dell", choose the first one
cleaned_df['Brand'] = cleaned_df['Brand'].str.replace(r'.* ?/ ?(.*)', r'\1', regex=True)

# Replace any value in the 'Brand' column that contains this format: "Brand + Version (or other properties)" with only brand names, eg. replace "Dell 16Gb" with "Dell"
patterns_to_replace = {
    r'Fuji.*': 'Fuji',
    r'Dell.*': 'Dell',
    r'Lenovo.*': 'Lenovo',
    r'Sony Vaio.*': 'Sony',
    r'Hp.*': 'Hp',
    r'Chimei.*': 'Chimei'
}

for pattern, replacement in patterns_to_replace.items():
    cleaned_df['Brand'] = cleaned_df['Brand'].str.replace(pattern, replacement, regex=True)

# Find the most frequent value in the 'brand' column
brand_most_frequent_value = cleaned_df['Brand'].mode()[0]

# Replace 'Unknown' with the most frequent value
cleaned_df['Brand'] = cleaned_df['Brand'].replace('Unknown', brand_most_frequent_value)

# Check the result
print(cleaned_df['Brand'].value_counts())

# Display the cleaned 'Brand' column
print(cleaned_df['Brand'].unique())


Brand
Dell       1947
Lenovo      786
Hp          524
Acer        157
Fuji        153
           ... 
Asem          1
Liteon        1
Vaio          1
Hyundai       1
Bormann       1
Name: count, Length: 84, dtype: int64
['Chuwi' 'Dell' 'Hp' 'Acer' 'Lenovo' 'Samsung' 'Microsoft' 'Lg' 'Toshiba'
 'Sgin' 'Panasonic' 'Asus' 'Olpc' 'Auusda' 'Fuji' 'Auo' 'Kodak'
 'Lcd Replacement' 'Packard Bell' 'Apple' 'Intel' 'Olidata' 'Jvc'
 'Simpletek' 'Chimei' 'Innolux' 'Getac' 'Google' 'BoeHydis' 'Compaq'
 'Hannstar' 'Delta' 'Hyundai' 'Fec' 'Au Optronics' 'Chunghwa' 'CPT'
 'Alienware' 'Vaio' 'Sony' 'Vector' 'Liteon' 'Ivo' 'Uniwill' 'Asem'
 'Zebra' 'Geo' 'Msi' 'Gateway' 'Hides' 'Gpd' 'Xplore' 'Craig' 'Huawei'
 'Metabox' 'Different' 'Kano' 'Ibm' 'Jumper' 'Media Wave' 'Durabook'
 'Gigabyte' 'Razer' 'Fusion5' 'Amiamo' 'Probook' 'Airbar' 'Zzetze'
 'Motion Computing' 'Goldbook' 'Insignia' 'Universal' 'Clevo' 'Nevada'
 'Hannnstar' 'Mixt' 'Gericom' 'Medion' 'Vulcan Electronics' 'Stone' 'Scsi'
 'Xnote' 'Iomega' 

In [6]:
#Clean processor column

# Browse all the values of processor
processor_values = cleaned_df['Processor'].unique()

# fill the blank 
cleaned_df['Processor'] = cleaned_df['Processor'].fillna('Unknown')

# Convert all processor names to lower case for consistency
cleaned_df['Processor'] = cleaned_df['Processor'].str.lower()

#Remove ® and ™ symbols from the 'Processor' column
cleaned_df['Processor'] = cleaned_df['Processor'].str.replace(r'[®™@]', '', regex=True)

#Replace any messy values in the 'Processor' column with "Unknown"
processor_to_replace = [
    '^?', '?','no','na','does not apply','not applicable','none','not specified','unspecified','quartz or automatic',
    'see title/description','see photos','dual core','mixt','backpack','cable','quad core'
]

cleaned_df['Processor'] = cleaned_df['Processor'].replace(processor_to_replace, 'Unknown')

# Correct the missspelling name
cleaned_df['Processor'] = cleaned_df['Processor'].str.replace(r'\b(?:inter|ntel|intle|intel)\b', 'Intel', regex=True)

# clean the Intel processor
cleaned_df['Processor'] = cleaned_df['Processor'].str.replace(r'(?i).*intel.*', 'Intel', regex=True)
cleaned_df['Processor'] = cleaned_df['Processor'].str.replace(r'.*\bi[\s-]?([0-9]).*', 'Intel', regex=True)
cleaned_df['Processor'] = cleaned_df['Processor'].str.replace(r'(?i).*(pentium|celeron).*', 'Intel', regex=True)
intel_to_replace = ['8365u','8265u','8250u','e3-1505m v6','n3530', '8650u','6300u','m7-6y75','fz-g1'
    
]

cleaned_df['Processor'] = cleaned_df['Processor'].replace(intel_to_replace, 'Intel')\

#clean the amd processor
cleaned_df['Processor'] = cleaned_df['Processor'].str.replace(r'(?i).*amd.*', 'AMD', regex=True)
amd_to_replace = ['radeon r4 dual-core','a10-8730b'
    
]

cleaned_df['Processor'] = cleaned_df['Processor'].replace(amd_to_replace, 'AMD')


#clean the qualcomm processor
cleaned_df['Processor'] = cleaned_df['Processor'].str.replace(r'(?i).*qualcomm.*', 'qualcomm', regex=True)

#clean the nvidia processor
cleaned_df['Processor'] = cleaned_df['Processor'].str.replace(r'(?i).*nvidia.*', 'nvidia', regex=True)

nvidia_to_replace = ['1660ti' 
    
]

cleaned_df['Processor'] = cleaned_df['Processor'].replace(nvidia_to_replace, 'nvidia')

#clean the android processor
cleaned_df['Processor'] = cleaned_df['Processor'].str.replace(r'(?i).*android.*', 'android', regex=True)

#clean the microsoft processor
cleaned_df['Processor'] = cleaned_df['Processor'].str.replace(r'(?i).*microsoft.*', 'microsoft', regex=True)

#clean the ryzen processor
cleaned_df['Processor'] = cleaned_df['Processor'].str.replace(r'(?i).*ryzen.*', 'ryzen', regex=True)

#clean the samsung processor
cleaned_df['Processor'] = cleaned_df['Processor'].str.replace(r'(?i).*samsung.*', 'samsung', regex=True)

#clean the apple processor
cleaned_df['Processor'] = cleaned_df['Processor'].str.replace(r'(?i).*apple.*', 'apple', regex=True)
apple_to_replace = ['a9 chip'
    
]

cleaned_df['Processor'] = cleaned_df['Processor'].replace(apple_to_replace, 'apple')

#clean the arm processor
cleaned_df['Processor'] = cleaned_df['Processor'].str.replace(r'(?i).*arm.*', 'arm', regex=True)
cleaned_df['Processor'] = cleaned_df['Processor'].str.replace('a72/a53 2.0ghz', 'arm')

#clean the rockchip processor
cleaned_df['Processor'] = cleaned_df['Processor'].str.replace(r'(?i).*rockchip.*', 'rockchip', regex=True)

#clean the mediatek processor
cleaned_df['Processor'] = cleaned_df['Processor'].str.replace(r'(?i).*mediatek.*', 'mediatek', regex=True)

#clean the google processor
cleaned_df['Processor'] = cleaned_df['Processor'].str.replace(r'(?i).*google.*', 'google', regex=True)

#clean the vivo processor
cleaned_df['Processor'] = cleaned_df['Processor'].str.replace(r'(?i).*v19.*', 'vivo', regex=True)

# Capitalize all the values in the 'Processor' column
cleaned_df['Processor'] = cleaned_df['Processor'].str.title()

print(cleaned_df['Processor'].unique())

# Find the most frequent value in the 'brand' column
Processor_most_frequent_value = cleaned_df['Processor'].mode()[0]

# Replace 'Unknown' with the most frequent value
cleaned_df['Processor'] = cleaned_df['Processor'].replace('Unknown', Processor_most_frequent_value)

# Check the result
print(cleaned_df['Processor'].value_counts())

# Display the cleaned 'Brand' column
print(cleaned_df['Processor'].unique())

['Unknown' 'Intel' 'Amd' 'Nvidia' 'Mediatek' 'Apple' 'Google' 'Qualcomm'
 'Vivo' 'Microsoft' 'Samsung' 'Arm' 'Android' 'Rockchip' 'Ryzen']
Processor
Intel        4475
Amd           163
Mediatek       23
Qualcomm       16
Apple           9
Microsoft       8
Samsung         6
Arm             6
Vivo            5
Nvidia          4
Google          3
Rockchip        2
Ryzen           2
Android         1
Name: count, dtype: int64
['Intel' 'Amd' 'Nvidia' 'Mediatek' 'Apple' 'Google' 'Qualcomm' 'Vivo'
 'Microsoft' 'Samsung' 'Arm' 'Android' 'Rockchip' 'Ryzen']


In [7]:
# clean color column

# Browse all the values of color
color_values = cleaned_df['Color'].unique()

# Convert all processor names to lower case for consistency
cleaned_df['Color'] = cleaned_df['Color'].str.lower()

# fill the blank 
cleaned_df['Color'] = cleaned_df['Color'].fillna('Unknown')

#Replace any messy values in the 'Color' column with "Unknown"
color_to_replace = [
    'gris','multi', 'see photos', 'see pictures','borgoña','negro','multicolor','immaculate','standard','multi-color','plata transparente'
]

cleaned_df['Color'] = cleaned_df['Color'].replace(color_to_replace, 'Unknown')

#any value contains silver will be converted to silver
cleaned_df['Color'] = cleaned_df['Color'].str.replace(r'(?i).*silver.*', 'silver', regex=True)

# If the color is not sure and have several choice such as "Black/Gray", choose the first one
cleaned_df['Color'] = cleaned_df['Color'].str.replace(r'.* ?/ ?(.*)', r'\1', regex=True)

#any value contains gray or grey will be converted to gray
cleaned_df['Color'] = cleaned_df['Color'].str.replace(r'(?i).*(gray|grey).*', 'gray', regex=True)

#any value contains brown will be converted to brown
cleaned_df['Color'] = cleaned_df['Color'].str.replace(r'(?i).*brown.*', 'brown', regex=True)

#any value contains black will be converted to black
cleaned_df['Color'] = cleaned_df['Color'].str.replace(r'(?i).*(black|blk).*', 'black', regex=True)

#any value contains blue will be converted to blue
cleaned_df['Color'] = cleaned_df['Color'].str.replace(r'(?i).*(blue|teal).*', 'blue', regex=True)

#any value contains platino will be converted to metal
cleaned_df['Color'] = cleaned_df['Color'].str.replace(r'(?i).*(platino|gun metal|dark metallic moon).*', 'metal', regex=True)

# Capitalize all the values in the 'Color' column
cleaned_df['Color'] = cleaned_df['Color'].str.title()

# Exclude 'Unknown' and find the most frequent color
most_frequent_color = cleaned_df[cleaned_df['Color'] != 'Unknown']['Color'].mode()[0]

# Replace 'Unknown' with the most frequent color (excluding 'Unknown')
cleaned_df['Color'] = cleaned_df['Color'].replace('Unknown', most_frequent_color)

# Check the result to ensure 'Unknown' values are replaced
print(cleaned_df['Color'].value_counts())


Color
Black        3802
Silver        437
Gray          311
Blue           79
White          22
Brown          11
Green          11
Metal          10
Red             9
Purple          8
Pink            5
Platinum        5
Beige           3
Orange          2
Bronze          2
Gold            2
Warm Gold       1
Yellow          1
Blanco          1
Sandstone       1
Name: count, dtype: int64


In [8]:
# clean screen size column

# Browse all the values of screen size
screen_values = cleaned_df['Screen Size'].unique()


#Replace any messy values in the 'Screen Size' column with "Unknown"
screen_to_replace = ['GLASSES SCREEN','DOES NOT APPLY', 'Laptop, in','none', 'Mixt','Not Applicable','Does not apply','?',
                     'WIDE','Various','See Title','Does Not Apply','Jumbo Heuer 1000'
    
]
cleaned_df['Screen Size'] = cleaned_df['Screen Size'].replace(screen_to_replace, 'Unknown')

# Remove any non-numeric characters from the 'Screen Size' column and keep only the numbers
cleaned_df['Screen Size'] = cleaned_df['Screen Size'].str.extract(r'(\d+\.?\d*)')

# fill the blank 
cleaned_df['Screen Size'] = cleaned_df['Screen Size'].fillna('Unknown')

# Exclude 'Unknown' and find the most frequent screen size
most_frequent_size = cleaned_df[cleaned_df['Screen Size'] != 'Unknown']['Screen Size'].mode()[0]

# Replace 'Unknown' with the most frequent Screen Size (excluding 'Unknown')
cleaned_df['Screen Size'] = cleaned_df['Screen Size'].replace('Unknown', most_frequent_size)

# Check the result to ensure 'Unknown' values are replaced
print(cleaned_df['Screen Size'].value_counts())


Screen Size
14       2605
15.6      581
11.6      312
15        166
13.3      162
13        159
12        139
12.5       85
17.3       64
13.5       59
14.1       52
16         49
10.1       46
12.3       36
10         25
17         23
15.4       22
11         21
12.1       11
7           9
15.5        8
8.1         7
10.5        7
16.0        6
9.7         6
18          6
14.0        6
13.4        5
10.8        5
7.2         5
11.5        3
18.4        3
4.5         3
2           3
10.95       3
16.1        3
6.1         2
14.5        2
10.3        2
10.2        2
13.2        2
12.4        1
14.2        1
13.1        1
12.2        1
9.5         1
13.9        1
10.4        1
13.7        1
Name: count, dtype: int64


In [9]:
# Clean type column
# Browse all the values of Type
type_values = cleaned_df['Type'].unique()

# fill the blank 
cleaned_df['Type'] = cleaned_df['Type'].fillna('Unknown')

# Convert all Type names to lower case for consistency
cleaned_df['Type'] = cleaned_df['Type'].str.lower()

#Replace any messy values in the 'Type' column with "unknown"
type_to_replace = ['awesome','nb','écran numériseur','loose key(s)','warranty: 90 days. warranty','dell',
                   'does not apply', 'getac','bureau'
]
cleaned_df['Type'] = cleaned_df['Type'].replace(type_to_replace, 'unknown')

# If the type is not sure and have several choice such as "notebook/laptop/tablet", choose the first one
cleaned_df['Type'] = cleaned_df['Type'].str.replace(r'.* ?/ ?(.*)', r'\1', regex=True)

#any value contains laptop will be converted to laptop
cleaned_df['Type'] = cleaned_df['Type'].str.replace(r'(?i).*laptop.*', 'laptop', regex=True)

#any value contains chromebook will be converted to chromebook
cleaned_df['Type'] = cleaned_df['Type'].str.replace(r'(?i).*chromebook.*', 'chromebook', regex=True)

#any value contains notebook will be converted to notebook
cleaned_df['Type'] = cleaned_df['Type'].str.replace(r'(?i).*notebook.*', 'notebook', regex=True)

#any value contains pc will be converted to pc
cleaned_df['Type'] = cleaned_df['Type'].str.replace(r'(?i).*pc.*', 'personal computer', regex=True)

#any value contains computer will be converted to computer
cleaned_df['Type'] = cleaned_df['Type'].str.replace(r'(?i).*computer.*', 'computer', regex=True)

#any value contains tablet will be converted to tablet
cleaned_df['Type'] = cleaned_df['Type'].str.replace(r'(?i).*tablet.*', 'tablet', regex=True)

#any value contains convertible will be convertible to tablet
cleaned_df['Type'] = cleaned_df['Type'].str.replace(r'(?i).*convertible.*', 'convertible', regex=True)

# Capitalize all the values in the 'Type' column
cleaned_df['Type'] = cleaned_df['Type'].str.title()

# Exclude 'Unknown' and find the most frequent type
most_frequent_type = cleaned_df[cleaned_df['Type'] != 'Unknown']['Type'].mode()[0]

# Replace 'Unknown' with the most frequent type (excluding 'Unknown')
cleaned_df['Type'] = cleaned_df['Type'].replace('Unknown', most_frequent_type)

# Check the result to ensure 'Unknown' values are replaced
print(cleaned_df['Type'].value_counts())


Type
Laptop                       4172
Computer                      250
Notebook                       73
Chromebook                     71
Tablet                         57
Netbook                        25
Ultrabook                      20
Wireless                        7
Ordinateur Portable             7
Toughbook                       6
Thinkpad                        5
Convertible                     4
Ultraportable                   4
Macbook Pro                     3
Lcd Touchscreen Assembly        3
Power Adapter                   3
Durabook                        2
Ebook Reader                    2
Desktop                         1
Performance Base Keyboard       1
Cardbus Adapter                 1
Keyboard                        1
Toughpad                        1
Portable Workstation            1
Ultra Rugged Toughbook          1
Lcd Display                     1
Scsi Cable                      1
Name: count, dtype: int64


In [10]:
# Clean model column
# Browse all the values of Model
model_values = cleaned_df['Model'].unique()
# fill the blank 
cleaned_df['Model'] = cleaned_df['Model'].fillna('Unknown')
# We found that values in the model column vary a lot, and this information is useful when a customer wants to retrieve, so we just leave it there.

In [11]:
#Clean the maximum resolution column

resolution_values = cleaned_df['Maximum Resolution'].unique()

# fill the blank 
cleaned_df['Maximum Resolution'] = cleaned_df['Maximum Resolution'].fillna('Unknown')

#Replace any messy values in the 'Maximum Resolution' column with "unknown"
resolution_to_replace = ['See Title/Description','ANY','Like New','HD', 'High Quality'
]
cleaned_df['Maximum Resolution'] = cleaned_df['Maximum Resolution'].replace(resolution_to_replace, 'Unknown')

# extract the format of "Number x Number" without other information
import re

def extract_resolution(res):
    match = re.search(r'(\d{3,4})\s*x\s*(\d{3,4})', res)
    if match:
        return f"{match.group(1)} x {match.group(2)}"
    return 'Unknown'

cleaned_df['Maximum Resolution'] = cleaned_df['Maximum Resolution'].apply(extract_resolution)


# Exclude 'Unknown' and find the most frequent type
most_frequent_resolution = cleaned_df[cleaned_df['Maximum Resolution'] != 'Unknown']['Maximum Resolution'].mode()[0]

# Replace 'Unknown' with the most frequent resolution (excluding 'Unknown')
cleaned_df['Maximum Resolution'] = cleaned_df['Maximum Resolution'].replace('Unknown', most_frequent_resolution)

# Check the result to ensure 'Unknown' values are replaced
print(cleaned_df['Maximum Resolution'].value_counts())



Maximum Resolution
1920 x 1080    3784
1366 x 768      522
1920 x 1200      86
2256 x 1504      47
1600 x 900       46
2560 x 1600      35
3840 x 2160      32
1280 x 800       29
2736 x 1824      20
2160 x 1440      17
1024 x 768       12
1920 x 1280      12
3000 x 2000      12
2880 x 1920      11
2560 x 1440       9
3840 x 2400       5
1024 x 600        4
3200 x 1800       3
2880 x 1800       3
1440 x 900        3
1366 x 912        3
3456 x 2160       3
3072 x 1920       3
1280 x 720        2
1800 x 1200       2
2200 x 1650       2
1900 x 1200       2
1024 x 480        2
2160 x 1350       1
1024 x 728        1
1360 x 768        1
1900 x 600        1
1366 x 786        1
1280 x 1024       1
1028 x 800        1
800 x 480         1
1680 x 1050       1
2220 x 1080       1
1368 x 788        1
1536 x 2048       1
Name: count, dtype: int64


In [12]:
#Clean the ram size column

ram_values = cleaned_df['Ram Size'].unique()

# fill the blank 
cleaned_df['Ram Size'] = cleaned_df['Ram Size'].fillna('Unknown')

#Replace any messy values in the 'Maximum Resolution' column with "unknown"
ram_values_to_replace = [
]
cleaned_df['Ram Size'] = cleaned_df['Ram Size'].replace(ram_values_to_replace, 'Unknown')


# Extract numbers from 'Ram Size' column and format them as "Number + GB"
def extract_ram_size(ram):
    match = re.search(r'(\d+)\s*GB', str(ram))
    if match:
        return f"{match.group(1)} GB"
    return 'Unknown'

cleaned_df['Ram Size'] = cleaned_df['Ram Size'].apply(extract_ram_size)

# Exclude 'Unknown' and find the most frequent type
most_frequent_ram = cleaned_df[cleaned_df['Ram Size'] != 'Unknown']['Ram Size'].mode()[0]

# Replace 'Unknown' with the most frequent ram size (excluding 'Unknown')
cleaned_df['Ram Size'] = cleaned_df['Ram Size'].replace('Unknown', most_frequent_ram)

# Check the result to ensure 'Unknown' values are replaced
print(cleaned_df['Ram Size'].value_counts())


ram_values = cleaned_df['Ram Size'].unique()
ram_values

Ram Size
8 GB      3216
16 GB      663
4 GB       527
32 GB      134
2 GB        46
12 GB       42
64 GB       26
3 GB        11
512 GB      11
6 GB         9
128 GB       8
256 GB       8
1 GB         7
24 GB        6
20 GB        4
40 GB        4
36 GB        1
Name: count, dtype: int64


array(['8 GB', '16 GB', '4 GB', '2 GB', '32 GB', '12 GB', '256 GB',
       '1 GB', '20 GB', '3 GB', '64 GB', '24 GB', '6 GB', '512 GB',
       '36 GB', '128 GB', '40 GB'], dtype=object)

In [13]:
#Clean the SSD Capacity column

# fill the blank 
cleaned_df['SSD Capacity'] = cleaned_df['SSD Capacity'].fillna('Unknown')

#Replace any messy values in the 'SSD Capacity' column with "unknown"
ssd_to_replace = ['See Options','Not Included', 'Not Applicable', 'Not Include','NONE', 'na', '0gb',
                  'No SSD','NO','1 Used,  1 support Max 1TB M.2 Extend','0','1TB Hard Drive (Not SSD)',
                  'FROM 120GB TILL 1TB SSD SELECT FROM ABOVE','Up to 2TB','M.2 120GB + HDD 1TB','Up to 1 TB',
                  '120GB - 1TB','120 GB - 240 GB','120 GB - 500 GB'
]
cleaned_df['SSD Capacity'] = cleaned_df['SSD Capacity'].replace(ssd_to_replace, 'Unknown')

# If the type is not sure and have several choice such as "256 GB / 512GB / 1TB SSD", choose the first one
cleaned_df['SSD Capacity'] = cleaned_df['SSD Capacity'].str.replace(r'.* ?/ ?(.*)', r'\1', regex=True)

# Function to clean SSD size into 'number + unit' format
def clean_ssd_size(ssd_value):
    
    # Use regex to find number and unit (GB, TB) in the SSD value
    match = re.search(r'(\d+)\s*(GB|TB)', ssd_value, re.IGNORECASE)
    
    if match:
        number = match.group(1)
        unit = match.group(2).upper()
        return f"{number} {unit}"
    
    # Return Unknown if no match is found
    return "Unknown"

# Apply the cleaning function to the 'SSD Capacity' column
cleaned_df['SSD Capacity'] = cleaned_df['SSD Capacity'].apply(clean_ssd_size)

# Exclude 'Unknown' and find the most frequent type
most_frequent_ssd = cleaned_df[cleaned_df['SSD Capacity'] != 'Unknown']['SSD Capacity'].mode()[0]

# Replace 'Unknown' with the most frequent SSD Capacity (excluding 'Unknown')
cleaned_df['SSD Capacity'] = cleaned_df['SSD Capacity'].replace('Unknown', most_frequent_ssd)

# Check the result to ensure 'Unknown' values are replaced
print(cleaned_df['SSD Capacity'].value_counts())



SSD Capacity
256 GB     2507
512 GB      469
240 GB      345
128 GB      262
120 GB      240
1 TB        158
16 GB       150
480 GB      146
960 GB      105
32 GB       103
2 TB         68
64 GB        68
500 GB       64
4 TB         11
360 GB        8
250 GB        5
4 GB          3
180 GB        3
476 GB        2
1024 GB       2
51 GB         1
800 GB        1
160 GB        1
115 GB        1
Name: count, dtype: int64


In [14]:
#Cleaning the OS Column
# Convert to lowercase and strip spaces
cleaned_df['OS'] = cleaned_df['OS'].str.lower().str.strip()

os_mapping = {
    r'.*windows 11 home.*': 'Windows 11 Home',
    r'.*windows 11 pro.*': 'Windows 11 Pro',
    r'.*windows 10 pro.*': 'Windows 10 Pro',
    r'.*windows 10 home.*': 'Windows 10 Home',
    r'.*windows 10.*': 'Windows 10',
    r'.*windows xp.*': 'Windows XP',
    r'.*windows vista.*': 'Windows Vista',
    r'.*windows 7.*': 'Windows 7',
    r'.*windows 8.1.*': 'Windows 8.1',
    r'.*windows 8.*': 'Windows 8',
    r'.*ubuntu.*': 'Ubuntu',
    r'.*freedos.*': 'FreeDOS',
    r'.*linux mint.*': 'Linux Mint',
    r'.*linux.*': 'Linux',
    r'.*chrome os.*': 'Chrome OS',
    r'.*google chrome os.*': 'Chrome OS',
    r'.*macos.*': 'macOS',
    r'.*mac os.*': 'macOS',
    r'.*android.*': 'Android',
    r'.*pop os.*': 'Pop!_OS',
    r'.*kali.*': 'Kali Linux',
    'not included': 'No OS',
    'no os': 'Unknown',
    'no operating system': 'No OS',
    'unknown': 'Unknown',
    'win 10 pro':'Windows 10 Pro',
    'window 11 pro' : 'Windows 11 Pro',
    'microsoft windows 11':'Windows 11'
}

# Apply the regex-based replacements for standardization
for pattern, replacement in os_mapping.items():
    cleaned_df['OS'] = cleaned_df['OS'].replace(pattern, replacement, regex=True)


# Handle NaN values by filling with 'Unknown'
cleaned_df['OS'] = cleaned_df['OS'].fillna('Unknown')

# If the type is not sure and have several choice choose the first one
cleaned_df['OS'] = cleaned_df['OS'].str.replace(r'.* ?/ ?(.*)', r'\1', regex=True)

# Capitalize all the values in the 'OS' column
cleaned_df['OS'] = cleaned_df['OS'].str.title()

# Replace 'Unknown' with the Most Frequent OS (excluding 'Unknown')
# Find the most frequent OS, excluding 'Unknown'
most_frequent_os = cleaned_df[cleaned_df['OS'] != 'Unknown']['OS'].mode()[0]

# Replace 'Unknown' with the most frequent OS
cleaned_df['OS'] = cleaned_df['OS'].replace('Unknown', most_frequent_os)


# Check the result to ensure 'Unknown' values are replace
print(cleaned_df['OS'].value_counts())

os_values = cleaned_df['OS'].unique()
os_values


OS
Windows 10 Pro                 2480
Windows 11 Pro                  802
Chrome Os                       338
Windows 10                      321
Windows 11 Home                 320
No Os                           142
Windows 11                       79
Windows 10 Home                  74
Windows 7                        47
Windows Xp                       27
Linux                            19
Macos                            13
Android                           9
Windows Home 11                   8
Windows Vista                     6
I5-8265U                          6
Ubuntu                            5
Windows 8.1                       5
Windows Pro                       3
Windows 11 S Mode                 3
Windows                           2
Freedos                           2
Win 11 Pro                        2
Windows10                         2
Win 10                            1
Windows11 Pro                     1
Windows 10 Pro 64                 1
Chrome                   

array(['Windows 11 Home', 'Windows 11 Pro', 'Windows 10 Pro', 'Chrome Os',
       'Windows 10', 'Win 11 Pro', 'No Os', 'Freedos', 'Windows 10 Home',
       'Windows Home 11', 'Windows Pro', 'Linux', 'Ubuntu', 'Windows',
       'Windows 11', 'Windows 7', 'I5-8265U', 'Android', 'Macos',
       'Windows Xp', 'Win 10', 'Windows 8.1', 'Windows11 Pro',
       'Windows 10 Pro 64', 'Chrome', 'Window 10 Pro Not Activated',
       'Kali Linux', 'Windows Vista', 'Windows 11 S Mode', 'Linux Mint',
       'Windows10', 'Pop!_Os'], dtype=object)

In [15]:
# Replace blank values or 'NO' with 'unknown' in the 'Hard Drive Capacity' column
cleaned_df['Hard Drive Capacity'] = cleaned_df['Hard Drive Capacity'].replace(['', 'NO'], 'unknown').fillna('unknown')

#Extract the number and unit from the 'Hard Drive Capacity' column
def clean_hard_drive_capacity(capacity):
    match = re.match(r"(\d+\.?\d*)\s*(GB|TB)", capacity, re.IGNORECASE)
    if match:
        number = match.group(1)
        unit = match.group(2).upper()
        return f"{number} {unit}"
    return capacity  # Return as is if no match

cleaned_df['Hard Drive Capacity'] = cleaned_df['Hard Drive Capacity'].apply(clean_hard_drive_capacity)

# Identify the most frequent capacity value (excluding 'unknown')
most_frequent_capacity = cleaned_df[cleaned_df['Hard Drive Capacity'] != 'unknown']['Hard Drive Capacity'].mode()[0]

# Replace all other unknown values (except 'unknown') with the most frequent value
cleaned_df['Hard Drive Capacity'] = cleaned_df['Hard Drive Capacity'].replace('unknown', most_frequent_capacity)

# Display the top few rows to verify the changes
cleaned_df['Hard Drive Capacity'].value_counts().head()

Hard Drive Capacity
256 GB    3473
512 GB     259
128 GB     138
16 GB      135
500 GB     124
Name: count, dtype: int64

In [16]:
# Remove any information in parentheses in the 'Storage Type' column
cleaned_df['Storage Type'] = cleaned_df['Storage Type'].str.replace(r"\(.*?\)", "", regex=True).str.strip()

# identify the most frequent storage type
most_frequent_storage_type = cleaned_df['Storage Type'].mode()[0]

# Replace any blank values with the most frequent storage type
cleaned_df['Storage Type'] = cleaned_df['Storage Type'].replace('', most_frequent_storage_type).fillna(most_frequent_storage_type)

# Display the top few rows to verify the changes
cleaned_df['Storage Type'].value_counts().head()


Storage Type
SSD             4161
HDD              225
eMMC             222
HDD + SSD         24
Not Included      12
Name: count, dtype: int64

In [17]:
# Replace 'nan' with 'unknown' in the 'Processor Speed' column
cleaned_df['Processor Speed'] = cleaned_df['Processor Speed'].replace('nan', 'unknown')

#Convert all values in the 'Processor Speed' column to strings
cleaned_df['Processor Speed'] = cleaned_df['Processor Speed'].astype(str)

# Extract number + unit and make format consistent
def extract_and_clean_processor_speed(speed):
    # Check for 'unknown'
    if speed == 'unknown':
        return speed
    # Extract number and unit (GHz or MHz) using regex
    match = re.match(r"(\d+\.?\d*)\s*(GHz|MHz)", speed, re.IGNORECASE)
    if match:
        number = match.group(1)
        unit = match.group(2).upper()
        return f"{number} {unit}"
    return 'unknown'  # If the value doesn't match the expected format, set it as 'unknown'

# Apply the cleaning function
cleaned_df['Processor Speed'] = cleaned_df['Processor Speed'].apply(extract_and_clean_processor_speed)

# Replace blank or 'unknown' values with the most frequent processor speed (excluding 'unknown')
most_frequent_speed = cleaned_df[cleaned_df['Processor Speed'] != 'unknown']['Processor Speed'].mode()[0]
cleaned_df['Processor Speed'] = cleaned_df['Processor Speed'].replace('unknown', most_frequent_speed)


# Display the top few rows to verify the changes
cleaned_df['Processor Speed'].value_counts().head()

Processor Speed
2.60 GHZ    2570
2.40 GHZ     230
2.80 GHZ     173
1.60 GHZ     160
1.10 GHZ     138
Name: count, dtype: int64

In [18]:
# Replace blank or NaN values with 'unknown' in the 'Features' column
cleaned_df['Features'] = cleaned_df['Features'].replace('', 'unknown').fillna('unknown')

# Display the top few rows to verify the changes
cleaned_df['Features'].value_counts().head()

Features
unknown                                     2052
Touchscreen                                  119
10/100 LAN Card, Wi-Fi, SD Card Reader       100
Wi-Fi                                         90
10/100 LAN Card,  Wi-Fi,  SD Card Reader      87
Name: count, dtype: int64

In [19]:
#Replace blank values with 'unknown' in the 'GPU' column
# GPU data vary a lot and might be useful for customers therefore we can just leave them there
cleaned_df['GPU'] = cleaned_df['GPU'].replace('', 'unknown').fillna('unknown')


In [20]:
# Adjusting the clean_price function to handle commas in prices
def clean_price(price):
    price = price.replace(',', '')  # Remove commas from the price
    if 'to' in price:  # Handle range prices by taking the average
        price_range = price.replace('$', '').split(' to ')
        return (float(price_range[0]) + float(price_range[1])) / 2
    else:
        return float(price.replace('$', '').strip())

# Apply the cleaning function to the Price column again
cleaned_df['Price'] = cleaned_df['Price'].apply(clean_price)

# Fill missing values with the mean of the 'Price' column
mean_price = cleaned_df['Price'].mean()
cleaned_df['Price'].fillna(mean_price, inplace=True)

/var/folders/94/9k2t7qs539579l97rq2fnr2w0000gn/T/ipykernel_58750/2123466100.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  cleaned_df['Price'].fillna(mean_price, inplace=True)


In [21]:
missing_value_counts = cleaned_df.isna().sum()
missing_value_percentages = missing_value_counts/len(cleaned_df) * 100
missing_value_percentages
cleaned_df

,Item Number,Brand,Processor,Screen Size,Color,Ram Size,SSD Capacity,GPU,Processor Speed,Type,Maximum Resolution,Model,OS,Hard Drive Capacity,Storage Type,Features,Condition,Price
0,85264981001,Chuwi,Intel,14,Gray,8 GB,256 GB,Intel Iris Plus Graphics 655,2.60 GHZ,Laptop,2160 x 1440,CoreBook X,Windows 11 Home,512 GB,SSD,"Backlit Keyboard, Built-in Microphone, Built...","New: A brand-new, unused, unopened, undamaged ...",303.68
1,85264981002,Dell,Intel,14,Black,8 GB,256 GB,Intel UHD Graphics 620,4.20 GHZ,Laptop,1920 x 1080,Dell Latitude 7490,Windows 11 Pro,2 TB,SSD,"Backlit Keyboard, Bluetooth, Built-in Micropho...",Very Good - RefurbishedThe item shows minimal ...,517.49
2,85264981003,Dell,Intel,14,Black,16 GB,500 GB,Intel HD Graphics,2.40 GHZ,Laptop,1920 x 1080,Dell Latitude E5470,Windows 10 Pro,500 GB,SSD,"10/100 LAN Card, Backlit Keyboard, Bluetooth, ...",UsedAn item that has been used previously. The...,175.00
3,85264981004,Hp,Intel,11.6,Black,4 GB,256 GB,Intel HD Graphics 500,2.40 GHZ,Laptop,1366 x 768,HP Chromebook 11 G6,Chrome Os,16 GB,eMMC,"Bluetooth, Built-in Microphone, Built-in Webca...",Good - RefurbishedThe item shows moderate wear...,84.99
4,85264981005,Dell,Intel,12.5,Black,8 GB,256 GB,Integrated,2.60 GHZ,Laptop,1366 x 768,Various Models,Windows 10,NO HDD,SSD,"10/100 LAN Card, Built-in Microphone, Built-in...",Good - RefurbishedThe item shows moderate wear...,101.22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6840,85264987613,Hp,Intel,14,Black,8 GB,256 GB,unknown,2.60 GHZ,Laptop,1920 x 1080,Unknown,Windows 10 Pro,256 GB,SSD,unknown,"New: A brand-new, unused, unopened, undamaged ...",90.92
6841,85264987614,Auo,Intel,14,Black,8 GB,256 GB,unknown,2.60 GHZ,Laptop,1920 x 1080,Unknown,Windows 10 Pro,256 GB,SSD,unknown,"New: A brand-new, unused, unopened, undamaged ...",116.97
6842,85264987615,Apple,Intel,14,Black,8 GB,256 GB,unknown,2.60 GHZ,Laptop,1920 x 1080,Unknown,Windows 10 Pro,256 GB,SSD,unknown,"New: A brand-new, unused, unopened, undamaged ...",316.05
6844,85264987617,Dell,Intel,14,Black,8 GB,256 GB,Gráficos Intel UHD 620,2.60 GHZ,Laptop,1920 x 1080,Unknown,Windows 10 Pro,256 GB,SSD,unknown,Seller refurbished: The item has been restored...,2108.99
